In [1]:
using Random
using DataFrames
using CSV
using Statistics
using Printf

struct Fighter
    name::String
    h::Int16
    f::Int16
    l::Int16
    t::Int16
end

const MXS = 400
const MXV = 20
const MNV = 2

function random_fighter(h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    flag = true
    while flag
        h = h0 + rand(0:(MXV - h0), 1)[1]
        f = f0 + rand(0:(MXV - f0), 1)[1]
        l = l0 + rand(0:(MXV - l0), 1)[1]
        t = t0 + rand(0:(MXV - t0), 1)[1]
        mm = minimum([h, f])
        budget = h*f+h*l+f*t
        if budget <= MXS && budget + mm > MXS
            flag = false
            return Fighter("No Name", h, f, l, t)
        end
    end
end

function eval_battle(a::Fighter, b::Fighter)::Int
    a_finds = a.f >= b.h
    b_finds = b.f >= a.h
    if a_finds && b_finds
        if a.t > b.t
            return 1
        end
        if a.t < b.t
            return -1
        end
        if a.t == b.t
            if a.l > b.l
                return 1
            end
            if a.l < b.l
                return -1
            end
            if a.l == b.l
                if a.h > b.h
                    return 1
                end
                if a.h < b.h
                    return -1
                end
                if a.h == b.h
                    return 0
                end
            end
        end
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        if a.l > b.l
            return 1
        end
        if a.l < b.l
            return -1
        end
        if a.l == b.l
            if a.t > b.t
                return 1
            end
            if a.t < b.t
                return -1
            end
            if a.t == b.t
                if a.f > b.f
                    return 1
                end
                if a.f < b.f
                    return -1
                end
                if a.f == b.f
                    return 0
                end
            end
        end
    end
end

function random_tournament_winner(c::Int, f::Function = random_fighter)::Fighter
    if c==0
        return f()
    end
    a = random_tournament_winner(c-1, f)
    b = random_tournament_winner(c-1, f)
    res = eval_battle(a, b)
    if res == 1
        return a
    else
        return b
    end
end


tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)

function random_name_and_stat()::Fighter
    vp = [0, 0, 0, 0]
    nametype = rand(1:5)
    name = ""
    if nametype == 1 || nametype == 2
        nm = rand(names)
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        if nametype == 1
            name = string(nm, " the ", adj)
        end
        if nametype == 2
            name = string(adj, " ", nm)
        end
    end
    if nametype == 3 || nametype == 4
        nm = rand(names)
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        if nametype == 3
            name = string(nm, " the ", noun)
        end
        if nametype == 4
            name = string(noun, " ", nm)
        end
    end
    if nametype == 5
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        name = string(adj, " ", noun)
    end
    Fighter(name, vp[1], vp[2], vp[3], vp[4])
end

function rand_rename(a::Fighter, n_tries::Int = 10)::Fighter
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.t^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.t * b.t)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Fighter(best_b.name, a.h, a.f, a.l, a.t)
end

function eval_battle_list(a::Fighter, bs::Array{Fighter})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function pick_best(as::Array{Fighter}, bs::Array{Fighter})::Fighter
    bestscore = -999
    bestf = as[1]
    for ii in 1:length(as)
        score = eval_battle_list(as[ii], bs)
        if score > bestscore
            bestscore = score
            bestf = as[ii]
        end
    end
    return bestf
end

function pick_best_rdmly(as::Array{Fighter}, bs::Array{Fighter}, ntries::Int)::Fighter
    bestscore = -999
    bestf = rand(as)
    for ii in 1:ntries
        f = rand(as)
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function fighters_to_df(as::Array{Fighter})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ts = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ts[ii] = as[ii].t    
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, t = ts)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

function eval_team_battle(as::Array{Fighter}, bs::Array{Fighter})::Int
    a_i = 1
    b_i = 1
    while (a_i <= length(as)) && (b_i <= length(bs))
        res = eval_battle(as[a_i], bs[b_i])
        if res == 1
            b_i = b_i + 1
        else
            a_i = a_i + 1
            if res == 0
                b_i = b_i + 1
            end
        end
    end
    a_out = (a_i > length(as))
    b_out = (b_i > length(as))
    if a_out
        if b_out
            return 0
        else
            return -1
        end
    else
        return 1
    end
end

eval_team_battle (generic function with 1 method)

In [2]:
function random_team(f::Function, team_size::Int)::Array{Fighter}
    team = Array{Fighter}(undef, team_size)
    for i in 1:team_size
        team[i] = f()
    end
    return team
end

random_team (generic function with 1 method)

In [3]:
library = Array{Fighter}(undef, 100000)
for i in 1:100000
    library[i] = rand_rename(random_fighter())
end

In [4]:
function evaluate_generator(f::Function, team_size::Int, n_times::Int = 1)::AbstractFloat
    scores = Array{AbstractFloat}(undef, n_times)
    for i0 in 1:n_times
        team = Array{Fighter}(undef, team_size)
        for i in 1:team_size
            team[i] = f()
        end
        bestscore = 999999
        exploiter = library[1]
        for i in 1:length(library)
            fighter = library[i]
            score = 0
            for j in 1:team_size
                score = score + eval_battle(team[j], fighter)
            end
            if score < bestscore
                bestscore = score
                exploiter = library[i]
            end
        end
        if n_times < 5
            println(exploiter)        
        end
        scores[i0] = bestscore/team_size
    end
    return mean(scores)
end

evaluate_generator (generic function with 2 methods)

In [5]:
function evaluate_generator2(f1, ts, np, limit)
    a_i = 1
    b_i = 1
    f_a = f1()
    t_a = random_team(f1, ts)
    f_b = pick_best_rdmly(library, t_a, np)
    while (a_i < limit) && (b_i < limit)
        res = eval_battle(f_a, f_b)
        if res != -1
            b_i = b_i + 1
            t_a = random_team(f1, ts)
            f_b = pick_best_rdmly(library, t_a, np)        
        end
        if res != 1
            a_i = a_i + 1
            f_a = f1()
        end
    end
    return (a_i/limit, b_i/limit)
end

evaluate_generator2 (generic function with 1 method)

In [6]:
# new random generation to more closely approximate Nash
function random_scheme_a(budget::Int = MXS, h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    budget_sofar = budget + 1
    name = ""
    while budget_sofar > budget
        hf_ratio = rand() * .75 + 0.02
        l_ratio = rand()
        h_temp = rand() + 0.07
        f_temp = rand() + 0.07
        name = string("Scheme-A (", @sprintf("%6.3f",hf_ratio), ",", @sprintf("%6.3f",l_ratio), ",", @sprintf("%6.3f",h_temp), ",", @sprintf("%6.3f",f_temp), ")")
        if hf_ratio > 0.05
            while abs(h_temp - f_temp) < 0.3
                h_temp = rand() + 0.07
                f_temp = rand() + 0.07
            end
        end
        budget_hf = (0.07 + hf_ratio * 0.8) * (budget * .75)
        h_ratio = h_temp/sqrt(h_temp * f_temp)
        f_ratio = f_temp/sqrt(h_temp * f_temp)
        h = max(h0, round(sqrt(budget_hf) * h_ratio))
        f = max(f0, round(sqrt(budget_hf) * f_ratio))
        budget_hf = h * f
        budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
        l_ratio2 = (h^2 + 5 * l_ratio)/(h^2+f^2 + 5)
        #l_ratio2 = (l_ratio * h)/(l_ratio * h + (1-l_ratio) * f)
        lraw = l0 + ( budget_remain * l_ratio2 )/h
        traw = t0 + ( budget_remain * (1 - l_ratio2) )/f
        l = trunc(lraw)
        t = trunc(traw)
        budget_sofar = h * f + h * l + f * t
        if (budget - budget_sofar) >= min(h, f)
            could_add_l = (budget - budget_sofar) >= h
            could_add_t = (budget - budget_sofar) >= f
            if could_add_l && !could_add_t
                l = l+trunc((budget - budget_sofar)/h)
            end
            if !could_add_l && could_add_t
                t = t+trunc((budget - budget_sofar)/f)
            end
            if could_add_l && could_add_t
                if fpart(lraw) > fpart(traw)
                    l = l+trunc((budget - budget_sofar)/h)
                else
                    t = t+trunc((budget - budget_sofar)/f)
                end
            end
        end
    end
    h = convert(Int, h)
    f = convert(Int, f)
    l = convert(Int, l)
    t = convert(Int, t)  
    return Fighter(name, h, f, l, t)
end

@time evaluate_generator2(random_scheme_a, 100, 100, 50000)

  8.463614 seconds (45.44 M allocations: 7.853 GiB, 13.87% gc time, 0.31% compilation time)


(0.69698, 1.0)

In [7]:
random_scheme_a()

Fighter("Scheme-A ( 0.146, 0.498, 0.835, 0.697)", 14, 4, 22, 9)

In [8]:
@time evaluate_generator(random_scheme_a, 2000, 5)

  9.397592 seconds (2.20 M allocations: 69.621 MiB, 0.20% gc time, 0.60% compilation time)


-0.368

In [9]:
budget = MXS
h0 = MNV
f0 = MNV
l0 = MNV
t0 = MNV
hf_ratio0 = 0
l_ratio0 = 0
h_temp0 = 0
f_temp0 = 0

hf_ratio = rand()
l_ratio = rand()
h_temp = rand()
f_temp = rand()
budget_hf = (0.1 + hf_ratio * 0.8) * (budget * .7)
h_ratio = h_temp/sqrt(h_temp * f_temp)
f_ratio = f_temp/sqrt(h_temp * f_temp)
h = max(h0, round(sqrt(budget_hf) * h_ratio))
f = max(f0, round(sqrt(budget_hf) * f_ratio))
budget_hf = h * f
budget_remain = (budget - budget_hf) - (l0 * h + t0 * f)
l_ratio2 = (h-2)^2/((h-2)^2+(f-2)^2)
#l_ratio2 = (l_ratio * h)/(l_ratio * h + (1-l_ratio) * f)
lraw = l0 + ( budget_remain * l_ratio2 )/h
traw = t0 + ( budget_remain * (1 - l_ratio2) )/f
l = trunc(lraw)
t = trunc(traw)
budget_sofar = h * f + h * l + f * t
if (budget - budget_sofar) >= min(h, f)
    could_add_l = (budget - budget_sofar) >= h
    could_add_t = (budget - budget_sofar) >= f
    if could_add_l && !could_add_t
        l = l+trunc((budget - budget_sofar)/h)
    end
    if !could_add_l && could_add_t
        t = t+trunc((budget - budget_sofar)/f)
    end
    if could_add_l && could_add_t
        if fpart(lraw) > fpart(traw)
            l = l+trunc((budget - budget_sofar)/h)
        else
            t = t+trunc((budget - budget_sofar)/f)
        end
    end
end

println((h, f, l, t, budget_hf, budget_sofar, hf_ratio))

(4.0, 36.0, 10.0, 6.0, 144.0, 368.0, 0.5211782043778418)


## Testing Random Scheme A

In [10]:
f = random_scheme_a()
print(f)
f.h * f.f + f.h * f.l + f.f * f.t

Fighter("Scheme-A ( 0.729, 0.961, 0.753, 0.916)", 11, 18, 5, 8)

397

In [11]:
rt = random_team(() -> random_scheme_a(), 1000)
fighters_to_df(rt)[1:10, :]

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,"Scheme-A ( 0.105, 0.102, 0.517, 0.121)",14,3,23,12
2,"Scheme-A ( 0.665, 0.641, 0.839, 0.913)",11,17,7,8
3,"Scheme-A ( 0.534, 0.822, 0.337, 0.695)",9,18,6,10
4,"Scheme-A ( 0.403, 0.335, 0.824, 0.585)",9,13,11,14
5,"Scheme-A ( 0.746, 0.440, 0.622, 0.522)",42,5,4,4
6,"Scheme-A ( 0.665, 0.356, 0.743, 0.237)",24,8,7,5
7,"Scheme-A ( 0.233, 0.329, 0.119, 0.973)",3,25,8,12
8,"Scheme-A ( 0.483, 0.739, 0.952, 0.563)",15,9,12,9
9,"Scheme-A ( 0.268, 0.318, 0.370, 0.906)",6,14,10,18


In [12]:
counters = random_team(() -> pick_best_rdmly(library, rt, 99), 100)
fighters_to_df(counters)[1:10, :]

Row,name,h,f,l,t
,String,Int64,Int64,Int64,Int64
1,Amazing Mallet,9,8,20,18
2,Powerful Reign,10,8,18,17
3,Fradel the Manic,10,8,18,17
4,Aven the Pizza,10,10,16,14
5,Classy Baboon,9,10,17,15
6,Pizza Avalee,10,10,14,16
7,Esa the Scissors,10,10,14,16
8,Minimal Pigeon,10,10,18,12
9,Nice Ox,10,9,17,15


In [13]:
rt = random_team(() -> random_scheme_a(), 100)

100-element Vector{Fighter}:
 Fighter("Scheme-A ( 0.417, 0.934, 0.834, 0.893)", 7, 18, 8, 12)
 Fighter("Scheme-A ( 0.216, 0.509, 0.506, 0.123)", 17, 4, 18, 6)
 Fighter("Scheme-A ( 0.277, 0.237, 0.529, 0.185)", 16, 6, 16, 8)
 Fighter("Scheme-A ( 0.663, 0.863, 0.563, 0.536)", 11, 17, 7, 8)
 Fighter("Scheme-A ( 0.432, 0.738, 0.714, 0.594)", 24, 5, 10, 8)
 Fighter("Scheme-A ( 0.260, 0.198, 0.868, 0.585)", 15, 5, 19, 8)
 Fighter("Scheme-A ( 0.278, 0.487, 0.331, 1.047)", 5, 17, 8, 16)
 Fighter("Scheme-A ( 0.633, 0.294, 0.709, 0.446)", 10, 17, 6, 10)
 Fighter("Scheme-A ( 0.087, 0.483, 0.499, 0.239)", 10, 4, 30, 15)
 Fighter("Scheme-A ( 0.646, 0.970, 0.673, 0.617)", 23, 8, 8, 4)
 Fighter("Scheme-A ( 0.286, 0.349, 0.614, 0.584)", 18, 5, 15, 8)
 Fighter("Scheme-A ( 0.563, 0.377, 0.470, 0.810)", 10, 16, 8, 10)
 Fighter("Scheme-A ( 0.025, 0.323, 0.730, 0.657)", 5, 5, 36, 39)
 ⋮
 Fighter("Scheme-A ( 0.331, 0.620, 0.577, 0.552)", 17, 6, 15, 7)
 Fighter("Scheme-A ( 0.572, 0.564, 0.585, 0.932)", 10, 1

In [14]:
for f in rt[1:20]
    v = eval_battle_list(f, counters)
    if v < 90
        println(f)
        println(v)
    end
end

Fighter("Scheme-A ( 0.417, 0.934, 0.834, 0.893)", 7, 18, 8, 12)
-98
Fighter("Scheme-A ( 0.216, 0.509, 0.506, 0.123)", 17, 4, 18, 6)
40
Fighter("Scheme-A ( 0.277, 0.237, 0.529, 0.185)", 16, 6, 16, 8)
-44
Fighter("Scheme-A ( 0.432, 0.738, 0.714, 0.594)", 24, 5, 10, 8)
-100
Fighter("Scheme-A ( 0.260, 0.198, 0.868, 0.585)", 15, 5, 19, 8)
70
Fighter("Scheme-A ( 0.278, 0.487, 0.331, 1.047)", 5, 17, 8, 16)
-36
Fighter("Scheme-A ( 0.633, 0.294, 0.709, 0.446)", 10, 17, 6, 10)
42
Fighter("Scheme-A ( 0.087, 0.483, 0.499, 0.239)", 10, 4, 30, 15)
42
Fighter("Scheme-A ( 0.646, 0.970, 0.673, 0.617)", 23, 8, 8, 4)
-98
Fighter("Scheme-A ( 0.286, 0.349, 0.614, 0.584)", 18, 5, 15, 8)
-54
Fighter("Scheme-A ( 0.563, 0.377, 0.470, 0.810)", 10, 16, 8, 10)
42
Fighter("Scheme-A ( 0.025, 0.323, 0.730, 0.657)", 5, 5, 36, 39)
-100
Fighter("Scheme-A ( 0.465, 0.068, 0.622, 0.592)", 7, 20, 5, 11)
-100
Fighter("Scheme-A ( 0.400, 0.531, 0.496, 0.742)", 5, 23, 6, 11)
-100
Fighter("Scheme-A ( 0.407, 0.068, 0.222, 1.062)

In [15]:
for f in counters[1:20]
    v = eval_battle_list(f, rt)
    if v < 90
        println(f)
        println(v)
    end
end

Fighter("Amazing Mallet", 9, 8, 20, 18)
30
Fighter("Powerful Reign", 10, 8, 18, 17)
28
Fighter("Fradel the Manic", 10, 8, 18, 17)
28
Fighter("Aven the Pizza", 10, 10, 16, 14)
30
Fighter("Classy Baboon", 9, 10, 17, 15)
32
Fighter("Pizza Avalee", 10, 10, 14, 16)
26
Fighter("Esa the Scissors", 10, 10, 14, 16)
26
Fighter("Minimal Pigeon", 10, 10, 18, 12)
36
Fighter("Nice Ox", 10, 9, 17, 15)
14
Fighter("Alexus the Cobbler", 10, 8, 18, 17)
28
Fighter("Joking Sphere", 9, 8, 19, 19)
32
Fighter("Whitney the Kitten", 10, 9, 17, 15)
14
Fighter("Icy Prankster", 10, 8, 17, 18)
26
Fighter("Egotistical Pizza", 10, 10, 14, 16)
26
Fighter("Egotistical Violet", 10, 9, 17, 15)
14
Fighter("Victorious Artisan", 11, 9, 14, 16)
12
Fighter("Armie the Phoenix", 10, 9, 15, 17)
16
Fighter("Giant Lukasey", 9, 9, 19, 16)
24
Fighter("Millie the Wizard", 10, 10, 17, 13)
34
Fighter("Xquisite Balloon", 10, 9, 19, 13)
20


In [16]:
for f in rt[1:20]
    v = eval_battle_list(f, counters)
    if v > 30
        println(f)
        println(v)
    end
end

Fighter("Scheme-A ( 0.216, 0.509, 0.506, 0.123)", 17, 4, 18, 6)
40
Fighter("Scheme-A ( 0.663, 0.863, 0.563, 0.536)", 11, 17, 7, 8)
98
Fighter("Scheme-A ( 0.260, 0.198, 0.868, 0.585)", 15, 5, 19, 8)
70
Fighter("Scheme-A ( 0.633, 0.294, 0.709, 0.446)", 10, 17, 6, 10)
42
Fighter("Scheme-A ( 0.087, 0.483, 0.499, 0.239)", 10, 4, 30, 15)
42
Fighter("Scheme-A ( 0.563, 0.377, 0.470, 0.810)", 10, 16, 8, 10)
42
Fighter("Scheme-A ( 0.707, 0.139, 1.027, 0.689)", 17, 11, 8, 7)
96
Fighter("Scheme-A ( 0.160, 0.611, 0.930, 0.415)", 12, 5, 23, 12)
98


In [17]:
evaluate_generator(random_scheme_a, 100, 20)

-0.47800000000000004

In [18]:
evaluate_generator(random_fighter, 100, 20)

-0.8

In [19]:
evaluate_generator(() -> random_tournament_winner(1), 100, 20)

-0.7699999999999998

In [20]:
evaluate_generator(() -> random_tournament_winner(2), 100, 20)

-0.7665

In [21]:
evaluate_generator(() -> random_tournament_winner(3), 100, 20)

-0.7235000000000001

In [22]:
evaluate_generator(() -> random_tournament_winner(4), 100, 20)

-0.6804999999999999

## Team battles

In [23]:
random_tournament_winner(2, random_scheme_a)

Fighter("Scheme-A ( 0.593, 0.915, 0.791, 1.020)", 25, 7, 8, 3)

In [24]:
team_a = random_team(() -> rand_rename(random_scheme_a()), 100)
team_a1 = random_team(() -> rand_rename(random_tournament_winner(1, random_scheme_a)), 100)
team_r0 = random_team(() -> rand_rename(random_tournament_winner(0)), 100)
team_r1 = random_team(() -> rand_rename(random_tournament_winner(1)), 100)
team_r2 = random_team(() -> rand_rename(random_tournament_winner(2)), 100)


100-element Vector{Fighter}:
 Fighter("Kulture the Zinc", 5, 15, 5, 20)
 Fighter("Spider Densley", 14, 19, 2, 5)
 Fighter("Adryana the Wise", 17, 15, 6, 2)
 Fighter("Phoenix Layci", 19, 4, 13, 19)
 Fighter("Junior Arik", 20, 5, 13, 8)
 Fighter("Alauna the Spider", 16, 17, 2, 5)
 Fighter("Slingshot Nadezhda", 13, 19, 4, 5)
 Fighter("Icosahedron Jerel", 11, 10, 10, 18)
 Fighter("Ahkeem the Iguana", 17, 14, 6, 4)
 Fighter("Wanita the Archer", 7, 15, 7, 16)
 Fighter("Jianni the Merciful", 20, 8, 10, 5)
 Fighter("Hot Cloud", 20, 3, 15, 13)
 Fighter("Richie the Tyro", 16, 8, 15, 4)
 ⋮
 Fighter("Emilian the Mushroom", 20, 6, 11, 10)
 Fighter("Lochlan the Kangaroo", 18, 10, 3, 16)
 Fighter("Hyena Leilan", 9, 20, 4, 9)
 Fighter("Junior Spider", 14, 15, 9, 4)
 Fighter("Rahman the Itinerant", 16, 13, 8, 4)
 Fighter("Balanced Damone", 18, 6, 14, 6)
 Fighter("Willful Sphere", 7, 14, 7, 18)
 Fighter("Satsuki the Zany", 15, 19, 2, 4)
 Fighter("Puppy Jeilanie", 12, 12, 17, 4)
 Fighter("Zemirah the Boo

In [25]:
eval_team_battle(team_a, counters)

1

In [26]:
eval_team_battle(team_a, team_r0)

1

In [27]:
eval_team_battle(team_a, team_r1)

1

In [28]:
eval_team_battle(team_a, team_r2)

1

In [29]:
eval_team_battle(team_a, team_a1)

-1

In [30]:
eval_team_battle(team_r0, team_a)

-1

In [31]:
eval_team_battle(team_r0, team_r1)

-1

In [32]:
eval_team_battle(team_r0, team_r2)

-1

In [33]:
as = team_a
bs = team_a1
a_i = 1
b_i = 1
print("         ")
println(as[a_i])
print("         ")
println(bs[b_i])
while (a_i <= length(as)) && (b_i <= length(bs))
    res = eval_battle(as[a_i], bs[b_i])
    if res == 1
        println(string(as[a_i].name, " won against ", bs[b_i].name))
        b_i = b_i + 1
        print("         ")
        println(bs[b_i])
    else
        if res == 0
            println(string(as[a_i].name, " tied against ", bs[b_i].name))
            b_i = b_i + 1
            print("         ")
            println(bs[b_i])
        else
            println(string(as[a_i].name, " lost against ", bs[b_i].name))
        end
        a_i = a_i + 1
        print("         ")
        println(as[a_i])
    end
end
a_out = (a_i > length(as))
b_out = (b_i > length(as))
if a_out
    if b_out
        return 0
    else
        return -1
    end
else
    return 1
end


         Fighter("Pug Asante", 8, 15, 8, 14)
         Fighter("Elevated Hyena", 5, 21, 4, 13)
Pug Asante won against Elevated Hyena
         Fighter("Blimp Beauty", 17, 7, 13, 8)
Pug Asante lost against Blimp Beauty
         Fighter("Zoah the Duck", 20, 10, 7, 6)
Zoah the Duck lost against Blimp Beauty
         Fighter("Rock December", 5, 21, 4, 13)
Rock December won against Blimp Beauty
         Fighter("Yuritzi the Nutritious", 12, 4, 25, 13)
Rock December won against Yuritzi the Nutritious
         Fighter("Xray Fareeha", 4, 16, 8, 19)
Rock December lost against Xray Fareeha
         Fighter("Nadelyn the Pangolin", 7, 21, 6, 10)
Nadelyn the Pangolin lost against Xray Fareeha
         Fighter("Massive Quilt", 8, 5, 31, 22)
Massive Quilt won against Xray Fareeha
         Fighter("Duck Najee", 18, 8, 11, 7)
Massive Quilt lost against Duck Najee
         Fighter("Agile Horse", 20, 10, 7, 6)
Agile Horse lost against Duck Najee
         Fighter("Wombat Kaylum", 9, 5, 28, 20)
Wombat Kaylum

         Fighter("Dorie the Rogue", 16, 4, 19, 8)
Cricket Philo lost against Dorie the Rogue
         Fighter("Martini Courtney", 17, 7, 13, 8)
Martini Courtney lost against Dorie the Rogue
         Fighter("Knight Hawthorn", 6, 25, 4, 9)
Knight Hawthorn won against Dorie the Rogue
         Fighter("Enthusiastic Warthog", 7, 18, 8, 12)
Knight Hawthorn lost against Enthusiastic Warthog
         Fighter("Aggressive Ladder", 3, 11, 12, 30)
Aggressive Ladder won against Enthusiastic Warthog
         Fighter("Militant Cube", 6, 14, 10, 18)
Aggressive Ladder won against Militant Cube
         Fighter("Cobbler Krista", 12, 5, 23, 12)
Aggressive Ladder lost against Cobbler Krista
         Fighter("Strong Blanket", 19, 9, 9, 6)
Strong Blanket lost against Cobbler Krista
         Fighter("Wombat Moncler", 10, 17, 6, 10)
Wombat Moncler won against Cobbler Krista
         Fighter("Timberly the Cat", 12, 6, 21, 12)
Wombat Moncler won against Timberly the Cat
         Fighter("Slingshot Kesiah", 7, 

LoadError: BoundsError: attempt to access 100-element Vector{Fighter} at index [101]

In [34]:
a_i, b_i

(101, 96)